## 긍정, 부정의 감성분석 : 데이터 양을 늘리기

In [1]:
import pandas as pd

# 두 파일을 병합하기
df_train = pd.read_table('ratings_train.txt')
df_test = pd.read_table('ratings_test.txt')

df = pd.concat([df_train, df_test])

In [2]:
# 중복 제거
df.drop_duplicates(subset=['document'], inplace=True)
# 결측치 제거
df = df.dropna(how='any')

In [3]:
# 중복과 결측치를 제거한 데이터를 파일로 저장하기
df.to_csv('naver_filtered.csv')

In [28]:
df = pd.read_csv("naver_filtered.csv")
# 데이터를 라벨 0,1의 비중을 맞춰서 1000를 추출

df = df[['document','label']]

df_pos = df[df['label'] == 1]
df_neg = df[df['label'] == 0]

sample_pos = df_pos.sample(n=3000, random_state=11)
sample_neg = df_neg.sample(n=3000, random_state=11)

sample_df = pd.concat([sample_pos,sample_neg], ignore_index=True)

print("샘플링된 데이터 개수 : ", len(sample_df))
print("라벨별 데이터 개수 : \n", sample_df['label'].value_counts())

df = sample_df

샘플링된 데이터 개수 :  6000
라벨별 데이터 개수 : 
 label
1    3000
0    3000
Name: count, dtype: int64


In [30]:
# 한글 형태소 분석기를 통해, 영화 리뷰 문장들을 전처리
# 한글만 남기고 모두 제거하기
df['document'] = df['document'].str.replace("[^가-힣 ]","",regex=True)
# '빈칸'만 남은 데이터는 공백으로 변경
df['document'] = df['document'].str.replace("^ +","",regex=True)
# 공백만 있는 데이터는 Null / Nan으로 변경
import numpy as np
df['document'] = df['document'].replace("", np.nan)

In [31]:
# df에서 null 제거
df = df.dropna(how='any')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 5963 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  5963 non-null   object
 1   label     5963 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 139.8+ KB
None


In [ ]:
stopwords = [
    '이', '가', '은', '는', '을', '를', '의', '에', '에서', '에게', '께', '로', '으로', 
    '와', '과', '보다', '처럼', '만큼', '같이', '까지', '마저', '조차', '부터', 
    '이나', '나', '이며', '며', '등', '하다', '한다', '하고', '하니', '하면', 
    '되어', '되다', '되고', '되니', '입니다', '습니다', 'ㅂ니다', '어요', '아요', '다', 
    '고', '면', '며', '게', '지', '죠',
    '그리고', '그러나', '하지만', '그런데', '그래서', '그러면', '그러므로', '따라서', 
    '또한', '또는', '및', '즉', '한편', '반면에', '근데',
    '나', '저', '우리', '저희', '너', '너희', '당신', '그', '그녀', '그들', '누구', 
    '무엇', '어디', '언제', '어느', '이것', '그것', '저것', '여기', '거기', '저기', 
    '이쪽', '그쪽', '저쪽',
    '하나', '둘', '셋', '넷', '다섯', '여섯', '일곱', '여덟', '아홉', '열',
    '일', '이', '삼', '사', '오', '육', '칠', '팔', '구', '십', '백', '천', '만',
    '첫째', '둘째', '셋째',
    '바로', '때', '것', '수', '일', '문제', '경우', '부분',
    '내용', '결과', '자체', '가지',
    '뿐', '대로', '만큼', '만', '지', '따름', '나름', '김에', '터', '너무', '어요'
    '아', '아이고', '아이구', '아하', '어', '그래', '응', '네', '예', '아니',
]

In [33]:
from konlpy.tag import Okt
okt = Okt()

# 훈련 데이터 만들기
train_data = []
for i in range(len(df)):
    emotion = df.iloc[i]['label']
    text = df.iloc[i]['document']
    # 문장을 형태소 분석하기 -> 문장을 단어(토큰)의 리스트로 변환 
    tokenized_text = okt.morphs(text)
    # 형태소 분석결과에서 불용어를 제거함
    filtered_tokens = [ word for word in tokenized_text if not word in stopwords ]
    # (토큰화 된 문장, 감정) 형태의 튜플을 리스트에 저장
    train_data.append((filtered_tokens,emotion))

In [34]:
all_tokens = []
for tokens, emotion in train_data :
    for token in tokens :
        all_tokens.append(token)
vocabulary = list(set(all_tokens))

In [35]:
def create_feature_set(tokens, vocabulary) :
    token_set = set(tokens)
    features = { word : (word in token_set) for word in vocabulary }
    return features

In [36]:
train_feature_datas = []
train_feature_datas = [
    (create_feature_set(tokens,vocabulary),emotion)
    for tokens, emotion in train_data ]

In [37]:
import nltk
model = nltk.NaiveBayesClassifier.train(train_feature_datas)

In [38]:
def classify_text(text, model, tokenizer, vocabulary) :
    tokens = tokenizer.morphs(text)
    filtered_tokens = [ word for word in tokens if not word in stopwords ]
    features = create_feature_set(filtered_tokens, vocabulary)
    clasified_label = model.classify(features)
    print(model.prob_classify(features).prob(clasified_label))
    result = "부정" if clasified_label == 0 else "긍정"
    print("예측 결과 : ", result)
    return result

In [ ]:
text_list = [
    "깨긋하구 아늑한분위기",
    "조용하고 주위에 볼거리도 많아요",
    "방이 작고 방음 안되요 주차장 찾기 어려워요",
    "3명이상 숙박하기에는 좋지 않습니다",
    "사장님도 친절하시고 숙소 안도 따뜻해서 좋았어요",
    "친절하고 깔끔해서 지내기 좋았습니다",
    "연박으로 묵었는데 크리스마스 너무 잘 보냈습니다 사장님 친절하시고 시설도 괜찮습니다",
    "친절하시고 위치도 경기전 근처라 한옥마을 다니기도 편했습니다",
    "딱 1박용으로 괜찮은 곳이에요",
    "특별한 서비스는 없었어요"
]
df = pd.read_csv('1_1400_output_partial.csv')

for text in df['sentence'] :
    classify_text(text, model, okt, vocabulary)

0.6049486528845978
예측 결과 :  긍정
0.7208121839324312
예측 결과 :  긍정
0.8881156728313495
예측 결과 :  부정
0.8311352534564015
예측 결과 :  부정
0.9544228812571377
예측 결과 :  긍정
0.969752393579962
예측 결과 :  긍정
0.9803166711836618
예측 결과 :  긍정
0.7723642590640472
예측 결과 :  긍정
0.9117733898121869
예측 결과 :  긍정
0.7245181372735994
예측 결과 :  부정


In [43]:
import pickle

# 모델 저장
with open("naive_bayes_model.pkl", "wb") as f:
    pickle.dump(model, f)

# 필요한 경우 단어 사전(vocabulary)도 같이 저장
with open("vocabulary.pkl", "wb") as f:
    pickle.dump(vocabulary, f)